
# Comparación de rendimiento entre MySQL y Cassandra

Este cuaderno proporciona instrucciones paso a paso y ejecuta directamente los scripts necesarios para cargar datos en MySQL y Cassandra, así como para realizar consultas.
Recuerda instalar todas las liberias necesarias antes de ejecutar los códigos, para ello en la terminal escribir "pip install nombre_libreria"


## Instrucciones para generar los datos

Ejecuta el siguiente código para generar el archivo `data.csv` con datos aleatorios:

In [ ]:
# Código para generar datos
import csv
from faker import Faker
import random
from tqdm import tqdm

NUM_RECORDS = 10000000
OUTPUT_FILE = 'data.csv'

fake = Faker()
emails = set()

with open(OUTPUT_FILE, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["name", "email", "age"])  # Header
    
    count = 0
    with tqdm(total=NUM_RECORDS, desc="Generando datos", unit="registros") as pbar:
        while count < NUM_RECORDS:
            email = fake.email()
            if email not in emails:  # Verifica si el email ya existe
                emails.add(email)
                writer.writerow([
                    fake.name(),
                    email,
                    random.randint(0, 99)
                ])
                count += 1
                pbar.update(1)  # Actualiza la barra de progreso

print(f'{NUM_RECORDS} registros únicos generados en el archivo {OUTPUT_FILE}.')

## Instrucciones para levantar servidor MySQL y cargar los datos


1. Asegúrate de que MySQL esté corriendo. Si usas Docker, ejecuta:
    ```bash
    docker-compose up -d mysql
    ```
2. Asegúrate de que la base de datos esté configurada correctamente.


In [ ]:

# Código para cargar datos en MySQL
import mysql.connector
import csv
from mysql.connector import Error

def insertar_datos():
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='user',
            password='password'

        )
        if conn.is_connected():
            print("Conexion creada")
            cursor = conn.cursor()

            # Verificar si la base de datos existe, si no, crearla
            cursor.execute("CREATE DATABASE IF NOT EXISTS mydatabase")
            print("Base de datos 'mydatabase' creada o ya existe")

            # Usar la base de datos
            cursor.execute("USE mydatabase")

            # Crear la tabla si no existe
# Crear tabla
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS users(
                    name VARCHAR(50) NOT NULL,
                    email VARCHAR(50),
                    age INT
                )
            """)
            print("tabla users creados")
#borrar todos los registros de la tabla usuarios para que no hayan copias
            cursor.execute("DELETE FROM users")
            conn.commit()
            print("registros anteriores eliminados")
            #leer datos del archivo csv
            with open('data.csv', mode='r') as archivo_csv:
                df = csv.reader(archivo_csv)
                next(df)             
                #insertar los datos
                for fila in df:
                    cursor.execute("""
                        INSERT INTO users(name, email, age)
                        values(%s,%s,%s)
                    """,fila)
            #confirmar los cambios
            conn.commit()
            print("Se han insertado los registros en la base de datos")

    except Error as e:
        print("error al conectar a la base de datos o al insertar los datos", e)
        
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("conexion cerrada")

insertar_datos()


## Instrucciones para levantar servidor Cassandra y cargar los datos


1. Asegúrate de que Cassandra esté corriendo. Si usas Docker, ejecuta:
    ```bash
    docker-compose up -d cassandra
    ```
2. Asegúrate de que la base de datos esté configurada correctamente.


In [ ]:

# Código para cargar datos en Cassandra
from cassandra.cluster import Cluster
import csv

cluster = Cluster(["localhost"])
session = cluster.connect()

# Crear keyspace y tabla
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS my_database
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };
""")
session.execute("USE my_database")
session.execute("""
    CREATE TABLE IF NOT EXISTS users (
        name TEXT,
        email TEXT PRIMARY KEY,
        age INT
    );
""")

# Insertar los datos desde el archivo CSV
with open("data.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        session.execute("""
            INSERT INTO users (name, email, age) VALUES (%s, %s, %s)
        """, (row["name"], row["email"], int(row["age"])))
cluster.shutdown()


## Instrucciones para consultar los datos en MySQL

Ejecuta el siguiente código para realizar las consultas en MySQL:

In [ ]:

# Código para realizar consultas en MySQL
import pymysql
import time

# Conexión
conn = pymysql.connect(host="localhost", user="user", password="password", database="mydatabase")
cursor = conn.cursor()

# Consulta 1: Promedio de edades
start_time = time.time()
cursor.execute("SELECT AVG(age) FROM users")
avg_age = cursor.fetchone()[0]
end_time = time.time()
print(f"Promedio de edad en MySQL: {avg_age}")
print(f"Tiempo de consulta (promedio de edades): {end_time - start_time:.4f} segundos")

# Consulta 2: Nombres que empiezan con 'R'
start_time = time.time()
cursor.execute("SELECT COUNT(*) FROM users WHERE name LIKE 'R%'")
count_names_with_r = cursor.fetchone()[0]
end_time = time.time()
print(f"Cantidad de nombres que empiezan con 'R' en MySQL: {count_names_with_r}")
print(f"Tiempo de consulta (nombres con 'R'): {end_time - start_time:.4f} segundos")


cursor.close()
conn.close()


## Instrucciones para consultar los datos en Cassandra

Ejecuta el siguiente código para realizar las consultas en Cassandra:

In [ ]:

# Código para realizar consultas en Cassandra
from cassandra.cluster import Cluster
import time

# Conexión
cluster = Cluster(["localhost"])
session = cluster.connect()
session.set_keyspace("my_database")

# Consulta 1: Promedio de edades
start_time = time.time()
rows = session.execute("SELECT age FROM users")
ages = [row.age for row in rows]
avg_age = sum(ages) / len(ages)
end_time = time.time()
print(f"Promedio de edad en Cassandra: {avg_age}")
print(f"Tiempo de consulta (promedio de edades): {end_time - start_time:.4f} segundos")

# Consulta 2: Nombres que empiezan con 'R'
start_time = time.time()
rows = session.execute("SELECT name FROM users")
count_names_with_r = sum(1 for row in rows if row.name.startswith('R'))
end_time = time.time()
print(f"Cantidad de nombres que empiezan con 'R' en Cassandra: {count_names_with_r}")
print(f"Tiempo de consulta (nombres con 'R'): {end_time - start_time:.4f} segundos")
